In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.MNIST(root='mnist_data', train=True, download = True, transform = transform)
test_data = datasets.MNIST(root='mnist_data', train=False, download = True, transform = transform)


train_loader = DataLoader(train_data, batch_size=64, shuffle = True)
test_loader = DataLoader(test_data, batch_size=64, shuffle = False)

In [ ]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model  = MNISTClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(model , train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        running_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

        print(f"epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")


In [ ]:
train(model,  train_loader, criterion, optimizer, epochs=20)

In [ ]:
def test(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100* correct / total}%")


test(model, test_loader)

In [ ]:
def show_predictions(model, test_loader):
    dataiter = iter(test_loader)
    images, labels = next(dataiter)

    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)


    fig, axes = plt.subplots(nrows=1, ncols=9, figsize=(10,2))
    for idx in range(9):
        ax = axes[idx]
        ax.imshow(images[idx]. squeeze(), cmap='gray')
        ax.set_title(f"Pred: {preds[idx]. item()}")
        ax.axis('off')
    plt.show()

show_predictions(model, test_loader)